In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import re, json, glob, argparse
from gensim.corpora import WikiCorpus, Dictionary
from gensim.utils import to_unicode

import argparse 
import json # 
import glob
import logging
import os
import random

import timeit
from tqdm import tqdm

import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler # Dataset을 보다 간편하게 사용하기 위한 함수
from torch.utils.data.distributed import DistributedSampler 
from fastprogress.fastprogress import master_bar, progress_bar # 학습 과정을 progress_bar로 예쁘게 표현하기 위해 사용하는 tool 
from torch.optim import Adam, AdamW
from torch.utils.data import TensorDataset, RandomSampler, DataLoader
#from transformers import BertTokenizer, BertTokenizerFast, BertForQuestionAnswering
#from transformers import get_linear_schedule_with_warmup

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import  f1_score

import gc

In [ ]:
memento=pd.read_excel('/content/drive/MyDrive/BIZ&AI_PAPER_11기/세희/memento/task/memento_1539.xlsx')
memento

,time,location,contexts
0,2022-06-26 01:43:56,정통집,석철이랑 제육볶음 먹었다 맛있었다
1,2021-11-18 05:25:00,설빙,가게 앞에서 귀여운 강아지를 봤다
2,2022-08-21 11:32:47,정통집,준상이랑 고기를 먹었다 맛있었다
3,2022-08-28 08:48:37,와플대학,딸기와플을 먹었다 저렴하고 맛있었다
4,2020-05-07 21:49:13,마쿠 마라탕,1시간 웨이팅하고 마라탕을 먹었다 사장님이 친절하셨다
...,...,...,...
1534,2020-01-15 17:28:23,힙덱,계란찜을 먹었다 별로 맛이 없었다
1535,2020-08-09 02:45:48,악어떡볶이,밥을 먹었다 지나가다가 세희를 만났다
1536,2020-09-04 21:42:11,롤링파스타,모르는 사람이랑 밥 먹었다 재미있었다
1537,2020-04-12 06:23:26,마쿠 마라탕,한양대점 버블티를 먹었다 알바생이 친절했다


preprocessing

In [ ]:
WIKI_REMOVE_CHARS = re.compile("'+|(=+.{2,30}=+)|__TOC__|(ファイル:).+|:(en|de|it|fr|es|kr|zh|no|fi):|\n", re.UNICODE)
WIKI_SPACE_CHARS = re.compile("(\\s|゙|゚|　)+", re.UNICODE)
EMAIL_PATTERN = re.compile("(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", re.UNICODE)
URL_PATTERN = re.compile("(ftp|http|https)?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", re.UNICODE)
WIKI_REMOVE_TOKEN_CHARS = re.compile("(\\*$|:$|^파일:.+|^;)", re.UNICODE)
MULTIPLE_SPACES = re.compile(' +', re.UNICODE)

In [ ]:
def tokenize(content, token_min_len=2, token_max_len=100, lower=True):
    content = re.sub(EMAIL_PATTERN, ' ', content)  # remove email pattern
    content = re.sub(URL_PATTERN, ' ', content) # remove url pattern
    content = re.sub(WIKI_REMOVE_CHARS, ' ', content)  # remove unnecessary chars
    content = re.sub(WIKI_SPACE_CHARS, ' ', content)
    content = re.sub(MULTIPLE_SPACES, ' ', content)
    content = re.sub(r'[.~,"\'-?:!;]', '', content)
    tokens = content.replace(", )", "").split(" ")

    result = []
    for token in tokens:
        if not token.startswith('_'):
            token_candidate = to_unicode(re.sub(WIKI_REMOVE_TOKEN_CHARS, '', token))
        else:
            token_candidate = ""
        if len(token_candidate) > 0: 
            result.append(token_candidate)
    return result

In [ ]:
memento['pre_text']=""

In [ ]:
memento['pre_text'] = memento['contexts'].apply(lambda text: tokenize(text))
memento['pre_text']

0                   [석철이랑, 제육볶음, 먹었다, 맛있었다]
1                  [가게, 앞에서, 귀여운, 강아지를, 봤다]
2                    [준상이랑, 고기를, 먹었다, 맛있었다]
3                  [딸기와플을, 먹었다, 저렴하고, 맛있었다]
4       [시간, 웨이팅하고, 마라탕을, 먹었다, 사장님이, 친절하셨다]
                       ...                 
1534               [계란찜을, 먹었다, 별로, 맛이, 없었다]
1535             [밥을, 먹었다, 지나가다가, 세희를, 만났다]
1536             [모르는, 사람이랑, 밥, 먹었다, 재미있었다]
1537          [한양대점, 버블티를, 먹었다, 알바생이, 친절했다]
1538       [동기모임을, 했다, 오랜만에, 친구를, 만나서, 좋았다]
Name: pre_text, Length: 1539, dtype: object

In [ ]:
memento

,time,location,contexts,pre_text
0,2022-06-26 01:43:56,정통집,석철이랑 제육볶음 먹었다 맛있었다,"[석철이랑, 제육볶음, 먹었다, 맛있었다]"
1,2021-11-18 05:25:00,설빙,가게 앞에서 귀여운 강아지를 봤다,"[가게, 앞에서, 귀여운, 강아지를, 봤다]"
2,2022-08-21 11:32:47,정통집,준상이랑 고기를 먹었다 맛있었다,"[준상이랑, 고기를, 먹었다, 맛있었다]"
3,2022-08-28 08:48:37,와플대학,딸기와플을 먹었다 저렴하고 맛있었다,"[딸기와플을, 먹었다, 저렴하고, 맛있었다]"
4,2020-05-07 21:49:13,마쿠 마라탕,1시간 웨이팅하고 마라탕을 먹었다 사장님이 친절하셨다,"[시간, 웨이팅하고, 마라탕을, 먹었다, 사장님이, 친절하셨다]"
...,...,...,...,...
1534,2020-01-15 17:28:23,힙덱,계란찜을 먹었다 별로 맛이 없었다,"[계란찜을, 먹었다, 별로, 맛이, 없었다]"
1535,2020-08-09 02:45:48,악어떡볶이,밥을 먹었다 지나가다가 세희를 만났다,"[밥을, 먹었다, 지나가다가, 세희를, 만났다]"
1536,2020-09-04 21:42:11,롤링파스타,모르는 사람이랑 밥 먹었다 재미있었다,"[모르는, 사람이랑, 밥, 먹었다, 재미있었다]"
1537,2020-04-12 06:23:26,마쿠 마라탕,한양대점 버블티를 먹었다 알바생이 친절했다,"[한양대점, 버블티를, 먹었다, 알바생이, 친절했다]"


In [ ]:
sentences = []
for context in memento["pre_text"] :
    sentence = ' '.join(context)
    sentence_test = re.sub("(\s){2,10}", "", sentence) #정규표현식으로 [\t\s\n\r\f 등 제거]
    sentences.append([sentence_test])
preprocess_data_df = pd.DataFrame(sentences, columns=['text'])
preprocess_data_df.tail()

,text
1534,계란찜을 먹었다 별로 맛이 없었다
1535,밥을 먹었다 지나가다가 세희를 만났다
1536,모르는 사람이랑 밥 먹었다 재미있었다
1537,한양대점 버블티를 먹었다 알바생이 친절했다
1538,동기모임을 했다 오랜만에 친구를 만나서 좋았다


In [ ]:
preprocess_data_df

,text
0,석철이랑 제육볶음 먹었다 맛있었다
1,가게 앞에서 귀여운 강아지를 봤다
2,준상이랑 고기를 먹었다 맛있었다
3,딸기와플을 먹었다 저렴하고 맛있었다
4,시간 웨이팅하고 마라탕을 먹었다 사장님이 친절하셨다
...,...
1534,계란찜을 먹었다 별로 맛이 없었다
1535,밥을 먹었다 지나가다가 세희를 만났다
1536,모르는 사람이랑 밥 먹었다 재미있었다
1537,한양대점 버블티를 먹었다 알바생이 친절했다


In [ ]:
memento_new = pd.concat([memento,preprocess_data_df], axis=1)
memento_new

,time,location,contexts,pre_text,text
0,2022-06-26 01:43:56,정통집,석철이랑 제육볶음 먹었다 맛있었다,"[석철이랑, 제육볶음, 먹었다, 맛있었다]",석철이랑 제육볶음 먹었다 맛있었다
1,2021-11-18 05:25:00,설빙,가게 앞에서 귀여운 강아지를 봤다,"[가게, 앞에서, 귀여운, 강아지를, 봤다]",가게 앞에서 귀여운 강아지를 봤다
2,2022-08-21 11:32:47,정통집,준상이랑 고기를 먹었다 맛있었다,"[준상이랑, 고기를, 먹었다, 맛있었다]",준상이랑 고기를 먹었다 맛있었다
3,2022-08-28 08:48:37,와플대학,딸기와플을 먹었다 저렴하고 맛있었다,"[딸기와플을, 먹었다, 저렴하고, 맛있었다]",딸기와플을 먹었다 저렴하고 맛있었다
4,2020-05-07 21:49:13,마쿠 마라탕,1시간 웨이팅하고 마라탕을 먹었다 사장님이 친절하셨다,"[시간, 웨이팅하고, 마라탕을, 먹었다, 사장님이, 친절하셨다]",시간 웨이팅하고 마라탕을 먹었다 사장님이 친절하셨다
...,...,...,...,...,...
1534,2020-01-15 17:28:23,힙덱,계란찜을 먹었다 별로 맛이 없었다,"[계란찜을, 먹었다, 별로, 맛이, 없었다]",계란찜을 먹었다 별로 맛이 없었다
1535,2020-08-09 02:45:48,악어떡볶이,밥을 먹었다 지나가다가 세희를 만났다,"[밥을, 먹었다, 지나가다가, 세희를, 만났다]",밥을 먹었다 지나가다가 세희를 만났다
1536,2020-09-04 21:42:11,롤링파스타,모르는 사람이랑 밥 먹었다 재미있었다,"[모르는, 사람이랑, 밥, 먹었다, 재미있었다]",모르는 사람이랑 밥 먹었다 재미있었다
1537,2020-04-12 06:23:26,마쿠 마라탕,한양대점 버블티를 먹었다 알바생이 친절했다,"[한양대점, 버블티를, 먹었다, 알바생이, 친절했다]",한양대점 버블티를 먹었다 알바생이 친절했다


In [ ]:
memento_new=memento_new.drop(['contexts','pre_text'], axis=1)
memento_new

,time,location,text
0,2022-06-26 01:43:56,정통집,석철이랑 제육볶음 먹었다 맛있었다
1,2021-11-18 05:25:00,설빙,가게 앞에서 귀여운 강아지를 봤다
2,2022-08-21 11:32:47,정통집,준상이랑 고기를 먹었다 맛있었다
3,2022-08-28 08:48:37,와플대학,딸기와플을 먹었다 저렴하고 맛있었다
4,2020-05-07 21:49:13,마쿠 마라탕,시간 웨이팅하고 마라탕을 먹었다 사장님이 친절하셨다
...,...,...,...
1534,2020-01-15 17:28:23,힙덱,계란찜을 먹었다 별로 맛이 없었다
1535,2020-08-09 02:45:48,악어떡볶이,밥을 먹었다 지나가다가 세희를 만났다
1536,2020-09-04 21:42:11,롤링파스타,모르는 사람이랑 밥 먹었다 재미있었다
1537,2020-04-12 06:23:26,마쿠 마라탕,한양대점 버블티를 먹었다 알바생이 친절했다


In [ ]:
text=[]
for i in memento_new['text']:
  if len(i)>6:
    text.append(i)

In [ ]:
text_df = pd.DataFrame(text, columns=['context'])
text_df

,context
0,석철이랑 제육볶음 먹었다 맛있었다
1,가게 앞에서 귀여운 강아지를 봤다
2,준상이랑 고기를 먹었다 맛있었다
3,딸기와플을 먹었다 저렴하고 맛있었다
4,시간 웨이팅하고 마라탕을 먹었다 사장님이 친절하셨다
...,...
1534,계란찜을 먹었다 별로 맛이 없었다
1535,밥을 먹었다 지나가다가 세희를 만났다
1536,모르는 사람이랑 밥 먹었다 재미있었다
1537,한양대점 버블티를 먹었다 알바생이 친절했다


In [ ]:
memento_new = pd.concat([memento_new,text_df], axis=1)
memento_new

,time,location,text,context
0,2022-06-26 01:43:56,정통집,석철이랑 제육볶음 먹었다 맛있었다,석철이랑 제육볶음 먹었다 맛있었다
1,2021-11-18 05:25:00,설빙,가게 앞에서 귀여운 강아지를 봤다,가게 앞에서 귀여운 강아지를 봤다
2,2022-08-21 11:32:47,정통집,준상이랑 고기를 먹었다 맛있었다,준상이랑 고기를 먹었다 맛있었다
3,2022-08-28 08:48:37,와플대학,딸기와플을 먹었다 저렴하고 맛있었다,딸기와플을 먹었다 저렴하고 맛있었다
4,2020-05-07 21:49:13,마쿠 마라탕,시간 웨이팅하고 마라탕을 먹었다 사장님이 친절하셨다,시간 웨이팅하고 마라탕을 먹었다 사장님이 친절하셨다
...,...,...,...,...
1534,2020-01-15 17:28:23,힙덱,계란찜을 먹었다 별로 맛이 없었다,계란찜을 먹었다 별로 맛이 없었다
1535,2020-08-09 02:45:48,악어떡볶이,밥을 먹었다 지나가다가 세희를 만났다,밥을 먹었다 지나가다가 세희를 만났다
1536,2020-09-04 21:42:11,롤링파스타,모르는 사람이랑 밥 먹었다 재미있었다,모르는 사람이랑 밥 먹었다 재미있었다
1537,2020-04-12 06:23:26,마쿠 마라탕,한양대점 버블티를 먹었다 알바생이 친절했다,한양대점 버블티를 먹었다 알바생이 친절했다


In [ ]:
memento_new=memento_new.drop(['text'], axis=1)
memento_new

,time,location,context
0,2022-06-26 01:43:56,정통집,석철이랑 제육볶음 먹었다 맛있었다
1,2021-11-18 05:25:00,설빙,가게 앞에서 귀여운 강아지를 봤다
2,2022-08-21 11:32:47,정통집,준상이랑 고기를 먹었다 맛있었다
3,2022-08-28 08:48:37,와플대학,딸기와플을 먹었다 저렴하고 맛있었다
4,2020-05-07 21:49:13,마쿠 마라탕,시간 웨이팅하고 마라탕을 먹었다 사장님이 친절하셨다
...,...,...,...
1534,2020-01-15 17:28:23,힙덱,계란찜을 먹었다 별로 맛이 없었다
1535,2020-08-09 02:45:48,악어떡볶이,밥을 먹었다 지나가다가 세희를 만났다
1536,2020-09-04 21:42:11,롤링파스타,모르는 사람이랑 밥 먹었다 재미있었다
1537,2020-04-12 06:23:26,마쿠 마라탕,한양대점 버블티를 먹었다 알바생이 친절했다


In [ ]:
memento_new=memento_new.dropna(axis=0)

In [ ]:
memento_new.columns=['time','location','document_text']

In [ ]:
memento_new

,time,location,document_text
0,2022-06-26 01:43:56,정통집,석철이랑 제육볶음 먹었다 맛있었다
1,2021-11-18 05:25:00,설빙,가게 앞에서 귀여운 강아지를 봤다
2,2022-08-21 11:32:47,정통집,준상이랑 고기를 먹었다 맛있었다
3,2022-08-28 08:48:37,와플대학,딸기와플을 먹었다 저렴하고 맛있었다
4,2020-05-07 21:49:13,마쿠 마라탕,시간 웨이팅하고 마라탕을 먹었다 사장님이 친절하셨다
...,...,...,...
1534,2020-01-15 17:28:23,힙덱,계란찜을 먹었다 별로 맛이 없었다
1535,2020-08-09 02:45:48,악어떡볶이,밥을 먹었다 지나가다가 세희를 만났다
1536,2020-09-04 21:42:11,롤링파스타,모르는 사람이랑 밥 먹었다 재미있었다
1537,2020-04-12 06:23:26,마쿠 마라탕,한양대점 버블티를 먹었다 알바생이 친절했다


data merge

In [ ]:
memento_new['date']=memento_new['time'].dt.strftime('%Y년 %m월 %d일')

In [ ]:
memento_new['times']=memento_new['time'].dt.strftime('%H시 %M분에')

In [ ]:
memento_new

,time,location,document_text,date,times
0,2022-06-26 01:43:56,정통집,석철이랑 제육볶음 먹었다 맛있었다,2022년 06월 26일,01시 43분에
1,2021-11-18 05:25:00,설빙,가게 앞에서 귀여운 강아지를 봤다,2021년 11월 18일,05시 25분에
2,2022-08-21 11:32:47,정통집,준상이랑 고기를 먹었다 맛있었다,2022년 08월 21일,11시 32분에
3,2022-08-28 08:48:37,와플대학,딸기와플을 먹었다 저렴하고 맛있었다,2022년 08월 28일,08시 48분에
4,2020-05-07 21:49:13,마쿠 마라탕,시간 웨이팅하고 마라탕을 먹었다 사장님이 친절하셨다,2020년 05월 07일,21시 49분에
...,...,...,...,...,...
1534,2020-01-15 17:28:23,힙덱,계란찜을 먹었다 별로 맛이 없었다,2020년 01월 15일,17시 28분에
1535,2020-08-09 02:45:48,악어떡볶이,밥을 먹었다 지나가다가 세희를 만났다,2020년 08월 09일,02시 45분에
1536,2020-09-04 21:42:11,롤링파스타,모르는 사람이랑 밥 먹었다 재미있었다,2020년 09월 04일,21시 42분에
1537,2020-04-12 06:23:26,마쿠 마라탕,한양대점 버블티를 먹었다 알바생이 친절했다,2020년 04월 12일,06시 23분에


In [ ]:
cols = ['date', 'times','location']
memento_new['dtl_combined'] =memento_new[cols].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
memento_new

,time,location,document_text,date,times,dtl_combined
0,2022-06-26 01:43:56,정통집,석철이랑 제육볶음 먹었다 맛있었다,2022년 06월 26일,01시 43분에,2022년 06월 26일 01시 43분에 정통집
1,2021-11-18 05:25:00,설빙,가게 앞에서 귀여운 강아지를 봤다,2021년 11월 18일,05시 25분에,2021년 11월 18일 05시 25분에 설빙
2,2022-08-21 11:32:47,정통집,준상이랑 고기를 먹었다 맛있었다,2022년 08월 21일,11시 32분에,2022년 08월 21일 11시 32분에 정통집
3,2022-08-28 08:48:37,와플대학,딸기와플을 먹었다 저렴하고 맛있었다,2022년 08월 28일,08시 48분에,2022년 08월 28일 08시 48분에 와플대학
4,2020-05-07 21:49:13,마쿠 마라탕,시간 웨이팅하고 마라탕을 먹었다 사장님이 친절하셨다,2020년 05월 07일,21시 49분에,2020년 05월 07일 21시 49분에 마쿠 마라탕
...,...,...,...,...,...,...
1534,2020-01-15 17:28:23,힙덱,계란찜을 먹었다 별로 맛이 없었다,2020년 01월 15일,17시 28분에,2020년 01월 15일 17시 28분에 힙덱
1535,2020-08-09 02:45:48,악어떡볶이,밥을 먹었다 지나가다가 세희를 만났다,2020년 08월 09일,02시 45분에,2020년 08월 09일 02시 45분에 악어떡볶이
1536,2020-09-04 21:42:11,롤링파스타,모르는 사람이랑 밥 먹었다 재미있었다,2020년 09월 04일,21시 42분에,2020년 09월 04일 21시 42분에 롤링파스타
1537,2020-04-12 06:23:26,마쿠 마라탕,한양대점 버블티를 먹었다 알바생이 친절했다,2020년 04월 12일,06시 23분에,2020년 04월 12일 06시 23분에 마쿠 마라탕


In [ ]:
memento_new['dtl_combined'] = memento_new['dtl_combined'].astype(str) + '에서 말했다 '

In [ ]:
cols = ['dtl_combined', 'document_text']
memento_new['dtlt_combined']=memento_new[cols].apply(lambda row: ''.join(row.values.astype(str)), axis=1)
memento_new

,time,location,document_text,date,times,dtl_combined,dtlt_combined
0,2022-06-26 01:43:56,정통집,석철이랑 제육볶음 먹었다 맛있었다,2022년 06월 26일,01시 43분에,2022년 06월 26일 01시 43분에 정통집에서 말했다,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...
1,2021-11-18 05:25:00,설빙,가게 앞에서 귀여운 강아지를 봤다,2021년 11월 18일,05시 25분에,2021년 11월 18일 05시 25분에 설빙에서 말했다,2021년 11월 18일 05시 25분에 설빙에서 말했다 가게 앞에서 귀여운 강아지...
2,2022-08-21 11:32:47,정통집,준상이랑 고기를 먹었다 맛있었다,2022년 08월 21일,11시 32분에,2022년 08월 21일 11시 32분에 정통집에서 말했다,2022년 08월 21일 11시 32분에 정통집에서 말했다 준상이랑 고기를 먹었다 ...
3,2022-08-28 08:48:37,와플대학,딸기와플을 먹었다 저렴하고 맛있었다,2022년 08월 28일,08시 48분에,2022년 08월 28일 08시 48분에 와플대학에서 말했다,2022년 08월 28일 08시 48분에 와플대학에서 말했다 딸기와플을 먹었다 저렴...
4,2020-05-07 21:49:13,마쿠 마라탕,시간 웨이팅하고 마라탕을 먹었다 사장님이 친절하셨다,2020년 05월 07일,21시 49분에,2020년 05월 07일 21시 49분에 마쿠 마라탕에서 말했다,2020년 05월 07일 21시 49분에 마쿠 마라탕에서 말했다 시간 웨이팅하고 마...
...,...,...,...,...,...,...,...
1534,2020-01-15 17:28:23,힙덱,계란찜을 먹었다 별로 맛이 없었다,2020년 01월 15일,17시 28분에,2020년 01월 15일 17시 28분에 힙덱에서 말했다,2020년 01월 15일 17시 28분에 힙덱에서 말했다 계란찜을 먹었다 별로 맛이...
1535,2020-08-09 02:45:48,악어떡볶이,밥을 먹었다 지나가다가 세희를 만났다,2020년 08월 09일,02시 45분에,2020년 08월 09일 02시 45분에 악어떡볶이에서 말했다,2020년 08월 09일 02시 45분에 악어떡볶이에서 말했다 밥을 먹었다 지나가다...
1536,2020-09-04 21:42:11,롤링파스타,모르는 사람이랑 밥 먹었다 재미있었다,2020년 09월 04일,21시 42분에,2020년 09월 04일 21시 42분에 롤링파스타에서 말했다,2020년 09월 04일 21시 42분에 롤링파스타에서 말했다 모르는 사람이랑 밥 ...
1537,2020-04-12 06:23:26,마쿠 마라탕,한양대점 버블티를 먹었다 알바생이 친절했다,2020년 04월 12일,06시 23분에,2020년 04월 12일 06시 23분에 마쿠 마라탕에서 말했다,2020년 04월 12일 06시 23분에 마쿠 마라탕에서 말했다 한양대점 버블티를 ...


In [ ]:
memento_new

,time,location,document_text,date,times,dtl_combined,dtlt_combined
0,2022-06-26 01:43:56,정통집,석철이랑 제육볶음 먹었다 맛있었다,2022년 06월 26일,01시 43분에,2022년 06월 26일 01시 43분에 정통집에서 말했다,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...
1,2021-11-18 05:25:00,설빙,가게 앞에서 귀여운 강아지를 봤다,2021년 11월 18일,05시 25분에,2021년 11월 18일 05시 25분에 설빙에서 말했다,2021년 11월 18일 05시 25분에 설빙에서 말했다 가게 앞에서 귀여운 강아지...
2,2022-08-21 11:32:47,정통집,준상이랑 고기를 먹었다 맛있었다,2022년 08월 21일,11시 32분에,2022년 08월 21일 11시 32분에 정통집에서 말했다,2022년 08월 21일 11시 32분에 정통집에서 말했다 준상이랑 고기를 먹었다 ...
3,2022-08-28 08:48:37,와플대학,딸기와플을 먹었다 저렴하고 맛있었다,2022년 08월 28일,08시 48분에,2022년 08월 28일 08시 48분에 와플대학에서 말했다,2022년 08월 28일 08시 48분에 와플대학에서 말했다 딸기와플을 먹었다 저렴...
4,2020-05-07 21:49:13,마쿠 마라탕,시간 웨이팅하고 마라탕을 먹었다 사장님이 친절하셨다,2020년 05월 07일,21시 49분에,2020년 05월 07일 21시 49분에 마쿠 마라탕에서 말했다,2020년 05월 07일 21시 49분에 마쿠 마라탕에서 말했다 시간 웨이팅하고 마...
...,...,...,...,...,...,...,...
1534,2020-01-15 17:28:23,힙덱,계란찜을 먹었다 별로 맛이 없었다,2020년 01월 15일,17시 28분에,2020년 01월 15일 17시 28분에 힙덱에서 말했다,2020년 01월 15일 17시 28분에 힙덱에서 말했다 계란찜을 먹었다 별로 맛이...
1535,2020-08-09 02:45:48,악어떡볶이,밥을 먹었다 지나가다가 세희를 만났다,2020년 08월 09일,02시 45분에,2020년 08월 09일 02시 45분에 악어떡볶이에서 말했다,2020년 08월 09일 02시 45분에 악어떡볶이에서 말했다 밥을 먹었다 지나가다...
1536,2020-09-04 21:42:11,롤링파스타,모르는 사람이랑 밥 먹었다 재미있었다,2020년 09월 04일,21시 42분에,2020년 09월 04일 21시 42분에 롤링파스타에서 말했다,2020년 09월 04일 21시 42분에 롤링파스타에서 말했다 모르는 사람이랑 밥 ...
1537,2020-04-12 06:23:26,마쿠 마라탕,한양대점 버블티를 먹었다 알바생이 친절했다,2020년 04월 12일,06시 23분에,2020년 04월 12일 06시 23분에 마쿠 마라탕에서 말했다,2020년 04월 12일 06시 23분에 마쿠 마라탕에서 말했다 한양대점 버블티를 ...


In [ ]:
memento_new.columns

Index(['time', 'location', 'document_text', 'date', 'times', 'dtl_combined',
       'dtlt_combined'],
      dtype='object')

In [ ]:
memento_new=memento_new.drop(['time','location','document_text','date','times','dtl_combined'], axis=1)
memento_new

,dtlt_combined
0,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...
1,2021년 11월 18일 05시 25분에 설빙에서 말했다 가게 앞에서 귀여운 강아지...
2,2022년 08월 21일 11시 32분에 정통집에서 말했다 준상이랑 고기를 먹었다 ...
3,2022년 08월 28일 08시 48분에 와플대학에서 말했다 딸기와플을 먹었다 저렴...
4,2020년 05월 07일 21시 49분에 마쿠 마라탕에서 말했다 시간 웨이팅하고 마...
...,...
1534,2020년 01월 15일 17시 28분에 힙덱에서 말했다 계란찜을 먹었다 별로 맛이...
1535,2020년 08월 09일 02시 45분에 악어떡볶이에서 말했다 밥을 먹었다 지나가다...
1536,2020년 09월 04일 21시 42분에 롤링파스타에서 말했다 모르는 사람이랑 밥 ...
1537,2020년 04월 12일 06시 23분에 마쿠 마라탕에서 말했다 한양대점 버블티를 ...


In [ ]:
memento_new.columns=['document_text']
memento_new

,document_text
0,2022년 06월 26일 01시 43분에 정통집에서 말했다 석철이랑 제육볶음 먹었다...
1,2021년 11월 18일 05시 25분에 설빙에서 말했다 가게 앞에서 귀여운 강아지...
2,2022년 08월 21일 11시 32분에 정통집에서 말했다 준상이랑 고기를 먹었다 ...
3,2022년 08월 28일 08시 48분에 와플대학에서 말했다 딸기와플을 먹었다 저렴...
4,2020년 05월 07일 21시 49분에 마쿠 마라탕에서 말했다 시간 웨이팅하고 마...
...,...
1534,2020년 01월 15일 17시 28분에 힙덱에서 말했다 계란찜을 먹었다 별로 맛이...
1535,2020년 08월 09일 02시 45분에 악어떡볶이에서 말했다 밥을 먹었다 지나가다...
1536,2020년 09월 04일 21시 42분에 롤링파스타에서 말했다 모르는 사람이랑 밥 ...
1537,2020년 04월 12일 06시 23분에 마쿠 마라탕에서 말했다 한양대점 버블티를 ...


In [ ]:
memento_new.to_csv("memento_final.csv", index = False)